In [1]:
!pip install nvidia-modulus nvidia-modulus-sym

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.1/220.1 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.1/772.1 kB 20.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.0/292.0 kB 21.7 MB/s eta 0:00:00
  Installing build dependencies ... don

In [2]:
import os
import warnings
import torch # (important for visualization part of the tutorial)
import matplotlib.pyplot as plt
import sympy
from sympy import Symbol, Eq, Abs


In [3]:
from modulus.sym.eq.pde import PDE
from sympy import symbols, diff, Matrix

class Maxwell2D(PDE):
    def __init__(self):
        # Define spatial and time variables
        x, y, t = symbols('x y t')

        # Define components of electric field (Ez) and magnetic field (Hx, Hy)
        E_z = symbols('E_z')
        H_x, H_y = symbols('H_x H_y')

        # Define permittivity (epsilon_0) and permeability (mu_0)
        epsilon_0, mu_0 = symbols('epsilon_0 mu_0')

        # Define the equations in 2D TM mode
        faraday_law_x = diff(H_x, t) + (1 / mu_0) * diff(E_z, y)
        faraday_law_y = diff(H_y, t) - (1 / mu_0) * diff(E_z, x)
        amperes_law = diff(E_z, t) - (1 / epsilon_0) * (diff(H_y, x) - diff(H_x, y))

        # Store the equations in self.equations
        self.equations = {
            'Faraday_Law_x': faraday_law_x,
            'Faraday_Law_y': faraday_law_y,
            'Amperes_Law': amperes_law
        }

        # Define output variables
        self.outputs = [E_z, H_x, H_y]


In [4]:
from modulus.sym.models.fully_connected import FullyConnectedArch
from modulus.sym.models.activation import Activation
from modulus.sym.key import Key

flow_net = FullyConnectedArch(
    input_keys=[Key("x"), Key("y")],
    output_keys=[Key("u"), Key("v"), Key("p")],
    layer_size=512,
    nr_layers=6,
    skip_connections=False,
    activation_fn=Activation.SILU,
    adaptive_activations=False,
    weight_norm=True
)

In [5]:
flow_net

FullyConnectedArch(
  (_impl): FullyConnectedArchCore(
    (layers): ModuleList(
      (0): FCLayer(
        (linear): WeightNormLinear(in_features=2, out_features=512, bias=True)
      )
      (1-5): 5 x FCLayer(
        (linear): WeightNormLinear(in_features=512, out_features=512, bias=True)
      )
    )
    (final_layer): FCLayer(
      (activation_fn): Identity()
      (linear): Linear(in_features=512, out_features=3, bias=True)
    )
  )
)

In [6]:
# Create an instance of Maxwell class
maxwell = Maxwell2D()

# Create nodes from Maxwell equations and other networks
nodes = maxwell.make_nodes() + [flow_net.make_node(name="flow_network")]


In [7]:
nodes

In [8]:
# from modulus.sym.geometry.primitives_2d import Rectangle
# from sympy import Symbol

# # Define the dimensions based on the MATLAB code
# height = 0.2  # in meters
# width = 0.2   # in meters

# # Create symbolic variables for x and y coordinates
# x, y = Symbol('x'), Symbol('y')

# # Define the rectangle geometry
# rec = Rectangle((-width/2, -height/2), (width/2, height/2))



In [9]:
# rec

In [10]:
from modulus.sym.domain import Domain

ldc_domain = Domain()

In [11]:
from modulus.sym.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseInteriorConstraint
)

In [12]:
from modulus.sym.geometry.primitives_2d import Rectangle
from sympy import Symbol

# Define the number of grid points and step sizes from the MATLAB code
NX = 200  # Number of points in x-direction
NY = 200  # Number of points in y-direction
dx = 0.001  # Spatial step in x-direction
dy = 0.001  # Spatial step in y-direction

# Calculate the total width and height of the domain
width = NX * dx  # Total width in meters
height = NY * dy  # Total height in meters

# Create symbolic variables for x and y coordinates
x, y = Symbol('x'), Symbol('y')

# Define the rectangle geometry based on the domain size
rec = Rectangle((-width/2, -height/2), (width/2, height/2))

In [13]:
rec

In [14]:
# Example boundary conditions after ensuring Ez, Hx, Hy are defined correctly

#Top wall at y = height
top_wall = PointwiseBoundaryConstraint(
    nodes= nodes,
    geometry=rec,
    outvar={
        "Ez": 0.0, "Hx": 0.0, "Hy": 0.0
    },
    batch_size=1000,
    lambda_weighting={"Ez": 1.0, "Hx": 1.0, "Hy": 1.0},
    criteria=(Eq(y, height))  # Adjusted criteria to match new rectangle definition
)

ldc_domain.add_constraint(top_wall, "top_wall")

# Bottom wall at y = 0
bottom_wall = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={
        "Ez": 0.0, "Hx": 0.0, "Hy": 0.0
    },
    batch_size=1000,
    lambda_weighting={"Ez": 1.0, "Hx": 1.0, "Hy": 1.0},
    criteria=(Eq(y, 0))  # Bottom wall at y = 0
)

ldc_domain.add_constraint(bottom_wall, "bottom_wall")

# Left wall at x = 0
left_wall = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={
        "Ez": 0.0, "Hx": 0.0, "Hy": 0.0
    },
    batch_size=1000,
    lambda_weighting={"Ez": 1.0, "Hx": 1.0, "Hy": 1.0},
    criteria=(Eq(x, 0))  # Left wall at x = 0
)

ldc_domain.add_constraint(left_wall, "left_wall")

# Right wall at x = width
right_wall = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={
        "Ez": 0.0, "Hx": 0.0, "Hy": 0.0
    },
    batch_size=1000,
    lambda_weighting={"Ez": 1.0, "Hx": 1.0, "Hy": 1.0},
    criteria=(Eq(x, width))  # Adjusted criteria to match new rectangle definition
)
ldc_domain.add_constraint(right_wall, "right_wall")




AssertionError: Geometry has no surface

In [ ]:
%%writefile config.yaml

defaults:
  - modulus_default  # Load default settings for Modulus
  - arch: fully_connected  # Use a fully connected architecture
  - scheduler: tf_exponential_lr  # Use TensorFlow exponential learning rate scheduler
  - optimizer: adam  # Use the Adam optimizer
  - loss: sum  # Loss function to sum contributions from each equation
  - _self_  # Include this configuration file itself

scheduler:
  decay_rate: 0.95  # Rate at which learning rate decays
  decay_steps: 4000  # Number of steps before decay is applied

training:
  rec_validation_freq: 1000  # Frequency of validation checks
  rec_inference_freq: 2000  # Frequency of running inference
  rec_monitor_freq: 1000  # Frequency of monitoring training performance
  rec_constraint_freq: 2000  # Frequency of checking constraints
  max_steps: 10000  # Maximum number of training steps

batch_size:
  TopWall: 1000  # Batch size for top wall boundary condition
  NoSlip: 1000  # Batch size for no-slip boundary condition
  Interior: 4000  # Batch size for interior points

graph:
  func_arch: true  # Whether to use a functional architecture for the graph

# Add any other relevant settings specific to your Maxwell's equations simulation
maxwell:
  epsilon_0: 8.854187817e-12  # Permittivity of free space
  mu_0: 1.256637061e-6  # Permeability of free space
  source_location: [50, 50]  # Location of the source in the grid
  monitor_location: [100, 150]  # Location of the monitor in the grid
  simulation_duration: 415  # Total time for simulation iterations
  spatial_step: 0.001  # Spatial step size
  time_step: 1.0e-9  # Time step size


In [ ]:
import torch
print(torch.__version__)

In [ ]:
pip install --upgrade modulus


In [ ]:
import modulus.sym
from modulus.sym.hydra import to_yaml
from modulus.sym.hydra.utils import compose
from modulus.sym.hydra.config import ModulusConfig


# Load the configuration from the specified YAML file
cfg = compose(config_path=".", config_name="config")

# Set the directory for saving network checkpoints
cfg.network_dir = 'outputs'

# Print the configuration in YAML format
print(to_yaml(cfg))

# # Additional settings specific to your Maxwell's equations simulation can be added here
# # For example, you could set additional parameters or modify the configuration
# cfg.maxwell.epsilon_0 = 8.854187817e-12  # Permittivity of free space
# cfg.maxwell.mu_0 = 1.256637061e-6  # Permeability of free space
# cfg.maxwell.source_location = [50, 50]  # Location of the source in the grid
# cfg.maxwell.monitor_location = [100, 150]  # Location of the monitor in the grid
# cfg.maxwell.simulation_duration = 415  # Total time for simulation iterations
# cfg.maxwell.spatial_step = 0.001  # Spatial step size
# cfg.maxwell.time_step = 1.0e-9  # Time step size

# # You can also save the modified configuration back to a YAML file if needed
with open("modified_config.yaml", "w") as f:
    f.write(to_yaml(cfg))
